# <font color=green>I. <U><i><b><strong>CODE PROVIDED</strong></b></i></U></font>

## Defining number of jobs and machines

In [1]:
N_JOBS = 7
N_MACHINES = 4

## function that generates random processing time

In [2]:
import random

def generateData(seed=0):
    PROC = [ [ 0 for m in range(N_MACHINES) ] for j in range(N_JOBS) ]
    random.seed(seed)
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            PROC[j][m] = random.randint(1,9)
    return PROC

## <font color=orange><U><i><b><strong>Runtime calculation function (implemented by me)</strong></b></i></U></font>

In [3]:
def calculate_runtime(start_time):
    end_time = time.time()
    total_runtime = end_time - start_time
    return total_runtime
import time
start_time = time.time()

## PROC generation

In [6]:
PROC = generateData(2821)

## Generating processing time as per seed 2821 from student ID 22242821

In [7]:
PROC

[[7, 3, 8, 8],
 [2, 4, 8, 3],
 [9, 7, 2, 9],
 [5, 2, 3, 4],
 [7, 7, 3, 3],
 [9, 3, 4, 3],
 [6, 9, 4, 6]]

## function to get integer value of PuLP variable

In [8]:
import pulp

In [9]:
def val(x):
    return int(pulp.value(x))

## linear programming problem to reduce job completion time

In [10]:
prob = pulp.LpProblem("JobScheduling",pulp.LpMinimize)

`J[j]` defines the sequencing of jobs 'j' and is just a permutation of the job number:

## decision variables for job scheduling

In [11]:
JJ = pulp.LpVariable.dicts("J", (range(N_JOBS), range(N_JOBS)),
                           lowBound=0, upBound=1, cat='Integer')

## function to find job corresponding to a given index

In [12]:
def job(n):
    for j in range(N_JOBS):
        if val(JJ[n][j])==1:
            return j

## function to generate job sequence based on LP solution

In [13]:
def jobSequence():
    acc = []
    for j in range(N_JOBS):
        for jj in range(N_JOBS):
            if pulp.value(JJ[j][jj])==1:
                acc.append(jj)
    return acc

## function for calculating processing time of a job on a machine

In [14]:
def proc(job, machine):
    global PROC
    return pulp.lpSum( [ PROC[j][machine] * JJ[j][job] for j in range(N_JOBS) ] ) 

`WAIT[j][m]` describes the wait time of job `j` before machine `m`

## descision variable for wait, idle, start and stop times

In [15]:
Wait = pulp.LpVariable.dicts("WAIT", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

`IDLE[j][m]` describes the idle time of machine `m` before processing job `j` 

In [16]:
Idle = pulp.LpVariable.dicts("IDLE", (range(N_JOBS), range(N_MACHINES)),
                             lowBound=0, cat='Integer')

`START[j][m]` describes the start time of machine `m` processing job `j`

In [17]:
Start = pulp.LpVariable.dicts("START", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

`STOP[j][m]` describes the stop time of machine `m` after processing job `j`

In [18]:
Stop = pulp.LpVariable.dicts("STOP", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

## Adding objective function to minimize completion time

In [19]:
prob += Stop[N_JOBS-1][N_MACHINES-1]

## constraints to ensure each job is scheduled exactly once

In [20]:
for j in range(N_JOBS):
    prob += pulp.lpSum( [ JJ[j][jj] for jj in range(N_JOBS) ] ) == 1
    prob += pulp.lpSum( [ JJ[jj][j] for jj in range(N_JOBS) ] ) == 1

## constraints to ensure jobs do not overlap on machines and respect processing times

In [21]:
for m in range(N_MACHINES):
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ Idle[ji][m] + proc(ji, m) for ji in range(j) ] ) + Idle[j][m] == Start[j][m]

## constraints to ensure jobs do not start until the previous job on the same machine is finished

In [22]:
for m in range(N_MACHINES):
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ Wait[j][mi] + proc(j, mi) for mi in range(m) ] ) + Wait[j][m] == Start[j][m]

## constraints to calculate stop times

In [23]:
for j in range(N_JOBS):
    for m in range(N_MACHINES):
        prob += Start[j][m] + proc(j,m) == Stop[j][m]

## Solve LP problem using the selected solver

In [24]:
solvers = pulp.listSolvers(onlyAvailable=True) 
solver = pulp.getSolver(solvers[0], msg=0)
prob.solve(solver)

1

## function to format and print the resulting job schedule

In [25]:
def schedule():
    row = ""
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            |" \
                f" Machine: {m:1d} |"
    row += '\n'
    for j in range(N_JOBS):
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Idle: {val(Idle[j][m]):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Start:{val(Start[j][m]):4d} |"  
        row += '\n' 
        row += f'| Job: {job(j):2d} |'
        for m in range(N_MACHINES):
            row += f" Wait: {val(Wait[j][m]):4d} | " \
                   f"Proc: {val(proc(j,m)):4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Stop: {val(Stop[j][m]):4d} |"  
        row += '\n' 
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    return row

The processing times per job and machine:

## Print the generated processing times

In [26]:
PROC

[[7, 3, 8, 8],
 [2, 4, 8, 3],
 [9, 7, 2, 9],
 [5, 2, 3, 4],
 [7, 7, 3, 3],
 [9, 3, 4, 3],
 [6, 9, 4, 6]]

The optimal job schedule:

## Print the generated job sequence

In [27]:
print(jobSequence())

[2, 0, 1, 6, 4, 5, 3]


The processing time of the optimal job schedule:

## Print the value of the objective function (completion time)

In [28]:
int(pulp.value(prob.objective))

54

In [29]:
makespan=int(pulp.value(prob.objective))
print("LP Makespan:",makespan)

LP Makespan: 54


The optimal job schedule in detail:

## Print the formatted job schedule

In [30]:
print(schedule())

|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    6 |            | Idle:   10 |            | Idle:   18 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   6 |            | Start:  10 |            | Start:  18 |
| Job:  2 | Wait:    0 | Proc:    2 | Wait:    4 | Proc:    4 | Wait:    0 | Proc:    8 | Wait:    0 | Proc:    3 |
|         |            | Stop:    2 |            | Stop:   10 |            | Stop:   18 |            | Stop:   21 |
|---------|------------|------------|------------|------------|---------

## <font color=orange><U><i><b><strong>Total runtime</strong></b></i></U></font>

In [41]:
total_runtime = calculate_runtime(start_time)
print(f"Total Runtime: {total_runtime:.6f} seconds")

Total Runtime: 0.231557 seconds


# -------------------------------------------------------------------------------------------------------------

# <font color=green>II. <U><i><b><strong>WRITTEN CODE</strong></b></i></U></font>

# Greedy 

## with makespan

In [43]:
import random
N_JOBS = 7
N_MACHINES = 4

def generateData(seed=0):
    PROC = [[0 for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    random.seed(seed)
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            PROC[j][m] = random.randint(1, 9)
    return PROC

def greedy_schedule(PROC):
    random.seed(2821)  
    machine_time = [0] * N_MACHINES
    job_sequence = []

    for _ in range(N_JOBS):
        min_time = float('inf')
        min_machine = None
        min_job = None
        
        for j in range(N_JOBS):
            if j not in job_sequence:
                for m in range(N_MACHINES):
                    if machine_time[m] + PROC[j][m] < min_time:
                        min_time = machine_time[m] + PROC[j][m]
                        min_machine = m
                        min_job = j
        
        job_sequence.append(min_job)
        machine_time[min_machine] = min_time
    
    return job_sequence

def print_job_schedule(PROC, job_sequence):
    starts = [[0] * N_MACHINES for _ in range(N_JOBS)]
    stops = [[0] * N_MACHINES for _ in range(N_JOBS)]

    row = "|---------|"
    for m in range(N_MACHINES):
        row += f"------------|------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            | Machine: {m:1d} |"
    row += '\n'
    for j in job_sequence:
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            if m == 0:
                starts[j][m] = stops[job_sequence[job_sequence.index(j) - 1]][m] if job_sequence.index(j) > 0 else 0
            else:
                starts[j][m] = max(stops[j][m - 1], stops[job_sequence[job_sequence.index(j) - 1]][m])
            stops[j][m] = starts[j][m] + PROC[j][m]
            idle = starts[j][m] - (stops[job_sequence[job_sequence.index(j) - 1]][m] if job_sequence.index(j) > 0 else 0)
            row += f"       {' ':4s} | Idle: {max(0, idle):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Start:{starts[j][m]:4d} |"
        row += '\n'
        row += f'| Job: {j:2d} |'
        for m in range(N_MACHINES):
            wait = starts[j][m] - (stops[j][m - 1] if m > 0 else 0)
            row += f" Wait: {max(0, wait):4d} | Proc: {PROC[j][m]:4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Stop: {stops[j][m]:4d} |"
        row += '\n'
    makespan_greedy = max(stop for job in stops for stop in job)    
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|------------|"
    row += '\n'
    row += f' Makespan value for Greedy: {makespan_greedy:4d}'
    
    return row

PROC = generateData(2821)
job_sequence = greedy_schedule(PROC)
print("Greedy Schedule-----")
print("Greedy Job Sequence:", job_sequence)
print(print_job_schedule(PROC, job_sequence))


Greedy Schedule-----
Greedy Job Sequence: [1, 2, 3, 4, 0, 5, 6]
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    2 |            | Idle:    6 |            | Idle:   14 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   2 |            | Start:   6 |            | Start:  14 |
| Job:  1 | Wait:    0 | Proc:    2 | Wait:    0 | Proc:    4 | Wait:    0 | Proc:    8 | Wait:    0 | Proc:    3 |
|         |            | Stop:    2 |            | Stop:    6 |            | Stop:   14 |            | Stop:   17 |
|-------

## showing runtime and makespan 

In [44]:
import random
import time
N_JOBS = 7
N_MACHINES = 4

def generateData(seed=0):
    PROC = [[0 for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    random.seed(seed)
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            PROC[j][m] = random.randint(1, 9)
    return PROC

def greedy_schedule(PROC):
    start_time = time.time()  
    random.seed(2821)  
    machine_time = [0] * N_MACHINES
    job_sequence = []

    for _ in range(N_JOBS):
        min_time = float('inf')
        min_machine = None
        min_job = None
        
        for j in range(N_JOBS):
            if j not in job_sequence:
                for m in range(N_MACHINES):
                    if machine_time[m] + PROC[j][m] < min_time:
                        min_time = machine_time[m] + PROC[j][m]
                        min_machine = m
                        min_job = j
        
        job_sequence.append(min_job)
        machine_time[min_machine] = min_time
    
    end_time = time.time()  
    runtime = end_time - start_time 
    return job_sequence, runtime

def print_job_schedule(PROC, job_sequence):
    starts = [[0] * N_MACHINES for _ in range(N_JOBS)]
    stops = [[0] * N_MACHINES for _ in range(N_JOBS)]

    row = "|---------|"
    for m in range(N_MACHINES):
        row += f"------------|------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            | Machine: {m:1d} |"
    row += '\n'
    for j in job_sequence:
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            if m == 0:
                starts[j][m] = stops[job_sequence[job_sequence.index(j) - 1]][m] if job_sequence.index(j) > 0 else 0
            else:
                starts[j][m] = max(stops[j][m - 1], stops[job_sequence[job_sequence.index(j) - 1]][m])
            stops[j][m] = starts[j][m] + PROC[j][m]
            idle = starts[j][m] - (stops[job_sequence[job_sequence.index(j) - 1]][m] if job_sequence.index(j) > 0 else 0)
            row += f"       {' ':4s} | Idle: {max(0, idle):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Start:{starts[j][m]:4d} |"
        row += '\n'
        row += f'| Job: {j:2d} |'
        for m in range(N_MACHINES):
            wait = starts[j][m] - (stops[j][m - 1] if m > 0 else 0)
            row += f" Wait: {max(0, wait):4d} | Proc: {PROC[j][m]:4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Stop: {stops[j][m]:4d} |"
        row += '\n'
    makespan_greedy = max(stop for job in stops for stop in job)    
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|------------|"
    row += '\n'
    row += f' Makespan value for Greedy: {makespan_greedy:4d}'
    
    return row

PROC = generateData(2821)
job_sequence, runtime_greedy = greedy_schedule(PROC)
print("Greedy Schedule----")
print("Greedy Job Sequence:", job_sequence)
print(print_job_schedule(PROC, job_sequence))
end_time = time.time()  
print(f"Greedy Algorithm Runtime value: {runtime_greedy:.6f} seconds")

Greedy Schedule----
Greedy Job Sequence: [1, 2, 3, 4, 0, 5, 6]
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    2 |            | Idle:    6 |            | Idle:   14 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   2 |            | Start:   6 |            | Start:  14 |
| Job:  1 | Wait:    0 | Proc:    2 | Wait:    0 | Proc:    4 | Wait:    0 | Proc:    8 | Wait:    0 | Proc:    3 |
|         |            | Stop:    2 |            | Stop:    6 |            | Stop:   14 |            | Stop:   17 |
|--------

In [61]:
def print_job_schedule(PROC, job_sequence):
    starts = [[0] * N_MACHINES for _ in range(N_JOBS)]
    stops = [[0] * N_MACHINES for _ in range(N_JOBS)]

    row = "|---------|"
    for m in range(N_MACHINES):
        row += f"------------|------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            | Machine: {m:1d} |"
    row += '\n'
    for j in job_sequence:
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            if m == 0:
                starts[j][m] = stops[job_sequence[job_sequence.index(j) - 1]][m] if job_sequence.index(j) > 0 else 0
            else:
                starts[j][m] = max(stops[j][m - 1], stops[job_sequence[job_sequence.index(j) - 1]][m])
            stops[j][m] = starts[j][m] + PROC[j][m]
            idle = starts[j][m] - (stops[job_sequence[job_sequence.index(j) - 1]][m] if job_sequence.index(j) > 0 else 0)
            row += f"       {' ':4s} | Idle: {max(0, idle):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Start:{starts[j][m]:4d} |"
        row += '\n'
        row += f'| Job: {j:2d} |'
        for m in range(N_MACHINES):
            wait = starts[j][m] - (stops[j][m - 1] if m > 0 else 0)
            row += f" Wait: {max(0, wait):4d} | Proc: {PROC[j][m]:4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Stop: {stops[j][m]:4d} |"
        row += '\n'
    makespan_greedy = max(stop for job in stops for stop in job)  
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|------------|"
    row += '\n'
    row += f' Makespan: {makespan_greedy:4d}'
    
    return makespan_greedy

PROC = generateData(2821)
makespan_greedy_value=print_job_schedule(PROC,job_sequence)
job_sequence = greedy_schedule(PROC)
print("Value of Greedy makespan:",makespan_greedy_value)
print("Greedy Job Sequence:", job_sequence)
print(print_job_schedule(PROC, job_sequence))

Value of Greedy makespan: 64
Greedy Job Sequence: [1, 2, 3, 4, 0, 5, 6]
64


## printing makespan value for greedy

In [52]:
print("Makespan value for Greedy:",makespan_greedy_value)

Makespan value for Greedy: 64


## printing PROC of Greedy

In [130]:
PROC = generateData(2821)
job_sequence = greedy_schedule(PROC)
print("Processing times (PROC) for the generated job sequence:")
for job_index in job_sequence:
    print(f"Job {job_index}: {PROC[job_index]}")

Processing times (PROC) for the generated job sequence:
Job 1: [2, 4, 8, 3]
Job 2: [9, 7, 2, 9]
Job 3: [5, 2, 3, 4]
Job 4: [7, 7, 3, 3]
Job 0: [7, 3, 8, 8]
Job 5: [9, 3, 4, 3]
Job 6: [6, 9, 4, 6]


# Stochastic

In [32]:
import random
import math
N_JOBS = 7
N_MACHINES = 4
INITIAL_TEMPERATURE = 1000
COOLING_RATE = 0.95
NUM_ITERATIONS = 1000

def generateData(seed=0):
    PROC = [[0 for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    random.seed(seed)
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            PROC[j][m] = random.randint(1, 9)
    return PROC

def initial_solution():
    solution = list(range(N_JOBS))
    random.shuffle(solution)
    return solution

def cost(solution, PROC):
    machine_time = [0] * N_MACHINES
    total_time = 0
    for job in solution:
        for machine in range(N_MACHINES):
            machine_time[machine] += PROC[job][machine]
        total_time += max(machine_time)
    return total_time

def neighbor(solution):
    neighbor_solution = solution.copy()
    idx1, idx2 = random.sample(range(N_JOBS), 2)
    neighbor_solution[idx1], neighbor_solution[idx2] = neighbor_solution[idx2], neighbor_solution[idx1]
    return neighbor_solution

def acceptance_probability(cost_current, cost_neighbor, temperature):
    if cost_neighbor < cost_current:
        return 1.0
    return math.exp((cost_current - cost_neighbor) / temperature)

def stochastic_search(PROC):
    current_solution = initial_solution()
    best_solution = current_solution
    temperature = INITIAL_TEMPERATURE
    while temperature > 1:
        for _ in range(NUM_ITERATIONS):
            neighbor_solution = neighbor(current_solution)
            current_cost = cost(current_solution, PROC)
            neighbor_cost = cost(neighbor_solution, PROC)
            if acceptance_probability(current_cost, neighbor_cost, temperature) > random.random():
                current_solution = neighbor_solution
                if neighbor_cost < cost(best_solution, PROC):
                    best_solution = neighbor_solution
        temperature *= COOLING_RATE
    return best_solution

def print_job_schedule(PROC, job_sequence):
    starts = [[0] * N_MACHINES for _ in range(N_JOBS)]
    stops = [[0] * N_MACHINES for _ in range(N_JOBS)]

    row = "|---------|"
    for m in range(N_MACHINES):
        row += f"------------|------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            | Machine: {m:1d} |"
    row += '\n'
    for j in job_sequence:
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            if m == 0:
                starts[j][m] = stops[job_sequence[job_sequence.index(j) - 1]][m] if job_sequence.index(j) > 0 else 0
            else:
                starts[j][m] = max(stops[j][m - 1], stops[job_sequence[job_sequence.index(j) - 1]][m])
            stops[j][m] = starts[j][m] + PROC[j][m]
            idle = starts[j][m] - (stops[job_sequence[job_sequence.index(j) - 1]][m] if job_sequence.index(j) > 0 else 0)
            row += f"       {' ':4s} | Idle: {max(0, idle):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Start:{starts[j][m]:4d} |"
        row += '\n'
        row += f'| Job: {j:2d} |'
        for m in range(N_MACHINES):
            wait = starts[j][m] - (stops[j][m - 1] if m > 0 else 0)
            row += f" Wait: {max(0, wait):4d} | Proc: {PROC[j][m]:4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Stop: {stops[j][m]:4d} |"
        row += '\n'
    makespan_stochastic = max(stop for job in stops for stop in job)    
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|------------|"
    row += '\n'
    row += f' Makespan for Stochastic: {makespan_stochastic:4d}'
    return row

PROC = generateData(2821)
job_sequence = stochastic_search(PROC)
print("Stochastic Search Schedule----")
print("Stochastic Job Sequence:", job_sequence)
print(print_job_schedule(PROC, job_sequence))


Stochastic Search Schedule----
Stochastic Job Sequence: [3, 1, 4, 0, 6, 5, 2]
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    5 |            | Idle:    7 |            | Idle:   10 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   5 |            | Start:   7 |            | Start:  10 |
| Job:  3 | Wait:    0 | Proc:    5 | Wait:    0 | Proc:    2 | Wait:    0 | Proc:    3 | Wait:    0 | Proc:    4 |
|         |            | Stop:    5 |            | Stop:    7 |            | Stop:   10 |            | Stop:  

## showing runtime and makespan

In [36]:
import random
import math
import time

N_JOBS = 7
N_MACHINES = 4
INITIAL_TEMPERATURE = 1000
COOLING_RATE = 0.95
NUM_ITERATIONS = 1000

def generateData(seed=0):
    PROC = [[0 for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    random.seed(seed)
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            PROC[j][m] = random.randint(1, 9)
    return PROC

def initial_solution():
    solution = list(range(N_JOBS))
    random.shuffle(solution)
    return solution

def cost(solution, PROC):
    machine_time = [0] * N_MACHINES
    total_time = 0
    for job in solution:
        for machine in range(N_MACHINES):
            machine_time[machine] += PROC[job][machine]
        total_time += max(machine_time)
    return total_time

def neighbor(solution):
    neighbor_solution = solution.copy()
    idx1, idx2 = random.sample(range(N_JOBS), 2)
    neighbor_solution[idx1], neighbor_solution[idx2] = neighbor_solution[idx2], neighbor_solution[idx1]
    return neighbor_solution

def acceptance_probability(cost_current, cost_neighbor, temperature):
    if cost_neighbor < cost_current:
        return 1.0
    return math.exp((cost_current - cost_neighbor) / temperature)

def stochastic_search(PROC):
    start_time = time.time()  
    current_solution = initial_solution()
    best_solution = current_solution
    temperature = INITIAL_TEMPERATURE
    while temperature > 1:
        for _ in range(NUM_ITERATIONS):
            neighbor_solution = neighbor(current_solution)
            current_cost = cost(current_solution, PROC)
            neighbor_cost = cost(neighbor_solution, PROC)
            if acceptance_probability(current_cost, neighbor_cost, temperature) > random.random():
                current_solution = neighbor_solution
                if neighbor_cost < cost(best_solution, PROC):
                    best_solution = neighbor_solution
        temperature *= COOLING_RATE
    end_time = time.time()  
    runtime = end_time - start_time  
    return best_solution, runtime

def print_job_schedule(PROC, job_sequence):
    starts = [[0] * N_MACHINES for _ in range(N_JOBS)]
    stops = [[0] * N_MACHINES for _ in range(N_JOBS)]

    row = "|---------|"
    for m in range(N_MACHINES):
        row += f"------------|------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            | Machine: {m:1d} |"
    row += '\n'
    for j in job_sequence:
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            if m == 0:
                starts[j][m] = stops[job_sequence[job_sequence.index(j) - 1]][m] if job_sequence.index(j) > 0 else 0
            else:
                starts[j][m] = max(stops[j][m - 1], stops[job_sequence[job_sequence.index(j) - 1]][m])
            stops[j][m] = starts[j][m] + PROC[j][m]
            idle = starts[j][m] - (stops[job_sequence[job_sequence.index(j) - 1]][m] if job_sequence.index(j) > 0 else 0)
            row += f"       {' ':4s} | Idle: {max(0, idle):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Start:{starts[j][m]:4d} |"
        row += '\n'
        row += f'| Job: {j:2d} |'
        for m in range(N_MACHINES):
            wait = starts[j][m] - (stops[j][m - 1] if m > 0 else 0)
            row += f" Wait: {max(0, wait):4d} | Proc: {PROC[j][m]:4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Stop: {stops[j][m]:4d} |"
        row += '\n'
    makespan_stochastic = max(stop for job in stops for stop in job)    
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|------------|"
    row += '\n'
    row += f' Makespan Value for Stochastic: {makespan_stochastic:4d}'
    return row

PROC = generateData(2821)
job_sequence, runtime_stochastic = stochastic_search(PROC)
print("Stochastic Search Schedule----")
print("Stochastic Job Sequence:", job_sequence)
print(print_job_schedule(PROC, job_sequence))
print(f"Stochastic Search Runtime: {runtime_stochastic:.6f} seconds")


Stochastic Search Schedule----
Stochastic Job Sequence: [3, 1, 4, 0, 6, 5, 2]
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    5 |            | Idle:    7 |            | Idle:   10 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   5 |            | Start:   7 |            | Start:  10 |
| Job:  3 | Wait:    0 | Proc:    5 | Wait:    0 | Proc:    2 | Wait:    0 | Proc:    3 | Wait:    0 | Proc:    4 |
|         |            | Stop:    5 |            | Stop:    7 |            | Stop:   10 |            | Stop:  

In [50]:
def print_job_schedule(PROC, job_sequence):
    starts = [[0] * N_MACHINES for _ in range(N_JOBS)]
    stops = [[0] * N_MACHINES for _ in range(N_JOBS)]

    row = "|---------|"
    for m in range(N_MACHINES):
        row += f"------------|------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            | Machine: {m:1d} |"
    row += '\n'
    for j in job_sequence:
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            if m == 0:
                starts[j][m] = stops[job_sequence[job_sequence.index(j) - 1]][m] if job_sequence.index(j) > 0 else 0
            else:
                starts[j][m] = max(stops[j][m - 1], stops[job_sequence[job_sequence.index(j) - 1]][m])
            stops[j][m] = starts[j][m] + PROC[j][m]
            idle = starts[j][m] - (stops[job_sequence[job_sequence.index(j) - 1]][m] if job_sequence.index(j) > 0 else 0)
            row += f"       {' ':4s} | Idle: {max(0, idle):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Start:{starts[j][m]:4d} |"
        row += '\n'
        row += f'| Job: {j:2d} |'
        for m in range(N_MACHINES):
            wait = starts[j][m] - (stops[j][m - 1] if m > 0 else 0)
            row += f" Wait: {max(0, wait):4d} | Proc: {PROC[j][m]:4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Stop: {stops[j][m]:4d} |"
        row += '\n'
    makespan_stochastic = max(stop for job in stops for stop in job)  
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|------------|"
    row += '\n'
    row += f' Makespan: {makespan_stochastic:4d}'
    
    return makespan_stochastic

PROC = generateData(2821)
makespan_stochastic_value=print_job_schedule(PROC,job_sequence)
job_sequence = stochastic_search(PROC)
print("Value of Stochastic makespan:",makespan_stochastic_value)
print("Stochastic Job Sequence:", job_sequence)
print(print_job_schedule(PROC, job_sequence))


Value of Stochastic makespan: 63
Stochastic Job Sequence: [3, 1, 4, 0, 6, 5, 2]
63


## showing makespan for stochastic

In [51]:
print("Makespan value for Stochastic:",makespan_stochastic_value)

Makespan value for Stochastic: 63


## Relative accuracy for Greedy

In [48]:
def calculate_relative_accuracy(makespan_greedy_value, makespan): 
    relative_accuracy = (1 - (makespan_greedy_value - makespan) / makespan) * 100 
    return relative_accuracy 

relative_accuracy = calculate_relative_accuracy(makespan_greedy_value, makespan) 
print("Greedy Relative Accuracy:", relative_accuracy, "%")


Greedy Relative Accuracy: 81.4814814814815 %


## Relative accuracy for Stochastic

In [49]:
def calculate_relative_accuracy_stochastic(makespan_stochastic_value, makespan): 
    relative_accuracy_stochastic = (1 - (makespan_stochastic_value - makespan) / makespan) * 100 
    return relative_accuracy_stochastic 

relative_accuracy_stochastic = calculate_relative_accuracy_stochastic(makespan_stochastic_value, makespan) 
print("Stochastic Relative Accuracy:", relative_accuracy_stochastic, "%")


Stochastic Relative Accuracy: 83.33333333333334 %


# ------------------------------------------------------------------------------------------------------------

## ----------------------------------- END ------------------------------------------------------------------------------